In [12]:
import os
import fitz
from tqdm import tqdm

In [13]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [24]:
zip_path = '統一'
count_true = 0
possible_rate = ['中立(維持評等)', '買進(調升評等)', '買進(維持評等)', '降低持股(調降評等)', '中立(調降評等)', '強力買進(調升評等)',
                    '強力買進(維持評等)', '未評等', '中立(初次評等)', '中立 (維持評等)', '強力買進(上調評等)', '買進(初次評等)', '買進 (維持評等)',
                    '買進(調降目標價)', '中立(降低評等)', '買進 (調升評等)', '買進', '賣出(調降評等)', '中立(調升評等)', '中立(下修評等)', 
                    '中立', '未評等(調整評等)', '中立 (調降評等)', '未評等(初次評等)', '強力買進(初次評等)']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        # 使用with語句讀取PDF文件，確保在使用後自動關閉文件。
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)  # 加載文檔的第0頁
            rect = page.rect
            # 檢查是否由元富投顧出版
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            if '統一證券投資顧問' in text_check_source :
                # 檢查版本
                clip_check_new_report = fitz.Rect(rect.width/2, 0, rect.width, 150)
                text_check_new_report = page.get_text(clip=clip_check_new_report)
                if '投資速報' in text_check_new_report :
                    # 報告為投資速報提取評價的第一種方法
                    try:
                        clip_old_report_1 = fitz.Rect(0, 0, 270, 270)
                        text_old_report_1 = page.get_text(clip=clip_old_report_1).strip()
                        text_old_report_1 = text_old_report_1.split('出刊緣由')[0].strip()
                        rate_1 = text_old_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                        # 報告為舊版個股報告 提取評價的第二種方法
                    try:
                        clip_old_report_2 = fitz.Rect(70, 130, 265, 160)
                        rate_2 = page.get_text(clip=clip_old_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                elif '訪談報告' in text_check_new_report :
                    # 報告為訪談報告提取評價的第一種方法
                    try:
                        clip_new_report_1 = fitz.Rect(375, 120, 565, 190)
                        text_new_report_1 = page.get_text(clip=clip_new_report_1).strip()
                        text_new_report_1 = text_new_report_1.split('出刊緣由')[0].strip()
                        rate_1 = text_new_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                        # 報告為訪談報提取評價的第二種方法
                    try:
                        clip_new_report_2 = fitz.Rect(375, 120, 565, 160)
                        rate_2 = page.get_text(clip=clip_new_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                elif '初次報告' in text_check_new_report :
                    # 報告為初次報告提取評價的第一種方法
                    try:
                        clip_new_report_1 = fitz.Rect(365, 190, 565, 380)
                        text_new_report_1 = page.get_text(clip=clip_new_report_1).strip()
                        text_new_report_1 = text_new_report_1.split('目標價')[0].strip()
                        rate_1 = text_new_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                    # 報告為初次報告提取評價的第二種方法
                    try:
                        clip_new_report_2 = fitz.Rect(365, 190, 565, 220)
                        rate_2 = page.get_text(clip=clip_new_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                elif '投資早報' in text_check_new_report or '港股投資' in text_check_new_report:
                    rate = 'wrong'
                else :
                    print(f'{text_check_new_report}, {directory_path} NULL')
            else:
                # False
                print(f'{directory_path} is not posted by 統一投顧')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # pass
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print(directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

 11%|█         | 303/2748 [00:09<01:33, 26.02it/s]

統一\2020-美亞-2020_03_31-統一投顧-NULL (2).pdf is not posted by 統一投顧
統一\2020-美亞-2020_03_31-統一投顧-NULL (2).pdf None rate_1=None, rate_2=None
統一\2020-美亞-2020_03_31-統一投顧-Null.pdf is not posted by 統一投顧
統一\2020-美亞-2020_03_31-統一投顧-Null.pdf None rate_1=None, rate_2=None


 32%|███▏      | 883/2748 [00:27<00:52, 35.61it/s]

統一\2637-慧洋-KY-統一投顧-中立-2637慧洋 統一 20230106.pdf is not posted by 統一投顧
統一\2637-慧洋-KY-統一投顧-中立-2637慧洋 統一 20230106.pdf None rate_1=None, rate_2=None


 55%|█████▍    | 1504/2748 [00:45<00:34, 36.28it/s]

...

統一\4726-永昕-2023_02_08-統一投顧-買進.pdf is not posted by 統一投顧
統一\4726-永昕-2023_02_08-統一投顧-買進.pdf None rate_1=None, rate_2=None


 95%|█████████▌| 2624/2748 [01:18<00:03, 36.64it/s]

統一\8464-億豐-2023_02_07-統一投顧-買進.pdf is not posted by 統一投顧
統一\8464-億豐-2023_02_07-統一投顧-買進.pdf None rate_1=None, rate_2=None


100%|██████████| 2748/2748 [01:22<00:00, 33.41it/s]

共2748件, 成功2742件, 失敗6件, 成功率:99.8%


In [15]:
import fitz
doc = fitz.open(r'統一\2338_光罩_2018-06-26_統一投顧_統一投顧-鄭高輯-【初次報告】光罩(2338.TT，買進) 收購美祿引進重要客戶，成熟製程光罩需求展望正向.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [16]:
# 檢查是否由統一投顧出版
page_check_source = doc.load_page(-1)
text_check_source = page_check_source.get_text()
if '統一證券投資顧問' in text_check_source :
    print('True')

True


In [17]:
# 檢查版本
clip_check_new_report = fitz.Rect(rect.width/2, 0, rect.width, 150)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_new_report)
pix.save("page-clip_check_new_report.png")
text_check_new_report = page.get_text(clip=clip_check_new_report)
if '投資速報' in text_check_new_report :
    print('投資速報')
elif '訪談報告' in text_check_new_report :
    print('訪談報告')
elif '初次報告' in text_check_new_report :
    print('初次報告')
elif '投資早報' in text_check_new_report or '港股投資' in text_check_new_report:
    print('投資早報 or 港股投資')
else :
    print('NULL')


初次報告


In [18]:
# 報告為舊版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(0, 0, 270, 270)
    text_old_report_1 = page.get_text(clip=clip_old_report_1).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_old_report_1.png")
    # text_old_report_1 = text_old_report_1.split('出刊緣由')[0].strip()
    rate_1 = text_old_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

'鄭高輯   OWENCHENG@uni-psg.com'

In [19]:
# 報告為舊版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(70, 130, 265, 160)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_old_report_2.png")
except:
    rate_2 = None
rate_2

'罩(2338.TT)'

In [20]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_new_report_1 = fitz.Rect(375, 120, 565, 190)
    text_new_report_1 = page.get_text(clip=clip_new_report_1).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_report_1)
    pix.save("page-clip_new_report_1.png")
    # text_new_report_1 = text_new_report_1.split('出刊緣由')[0].strip()
    rate_1 = text_new_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

''

In [21]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_new_report_2 = fitz.Rect(375, 120, 565, 160)
    rate_2 = page.get_text(clip=clip_new_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

''

In [22]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_new_report_1 = fitz.Rect(365, 190, 565, 380)
    text_new_report_1 = page.get_text(clip=clip_new_report_1).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_report_1)
    pix.save("page-clip_new_report_1.png")
    text_new_report_1 = text_new_report_1.split('目標價')[0].strip()
    rate_1 = text_new_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

''

In [23]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_new_report_2 = fitz.Rect(365, 190, 565, 220)
    rate_2 = page.get_text(clip=clip_new_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

'目標價  \nNT$37'